## Librerías

In [2]:
# Librerías de ciencia de datos 
import numpy as np
import pandas as pd
import sqlite3 as sql

# Librerías para preprocesamiento 
from sklearn.preprocessing import MinMaxScaler

# Librerías para análisis interactivo 
from ipywidgets import interact 
from sklearn import neighbors 
import joblib

# Paquete para sistemas de recomendación surprise 
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise.model_selection import train_test_split

## Carga de datos

In [3]:
#### Conectar #######
conn=sql.connect('data/db_movies')
cur=conn.cursor()

In [4]:
#ver tablas
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('movies_sel',),
 ('ratings_final',),
 ('movie_final',),
 ('full_ratings',)]

## 3 Sistema de recomendación basado en contenido KNN Con base en todo lo visto por el usuario

### Top 10 de las películas más similares a las vistas por el usuario y con una que le haya dado una calificación mayor a 4.

In [20]:
# Vamos a trabajar con el dataframe escalado y con dummies df_dum 
df_dum = pd.read_csv('data/df_dum.csv')
movies = pd.read_sql("SELECT * FROM movie_final", conn)

# Se selecciona un usuario para realizar las recomendaciones 
usuarios = pd.read_sql('SELECT distinct (userId) as user_id from ratings_final', conn)

# Seleccionamos un usuario para realizar el sistema de recomendación
user_id = 607
movies
ratings = pd.read_sql('SELECT * FROM ratings_final', conn)

In [21]:
usuarios

,user_id
0,1
1,4
2,6
3,7
4,10
...,...
369,605
370,606
371,607
372,608


In [29]:

def recomendar(user_id=list(usuarios['user_id'].value_counts().index)):
    
    # Se seleccionan los ratings del usuario establecido
    ratings = pd.read_sql('SELECT * from ratings_final where userId=:user', conn, params={'user': user_id})
    
    #Seleccionar solo peliculas con rating mayor a 4
    ratings = ratings[ratings['rating'] > 4]
    
    # Convertir ratings del usuario a array
    ratings_array = ratings['movieId'].to_numpy()
    
    # Agregar la columna de movieId y título al dataframe df_dum para filtrar y mostrar nombres
    df_dum[['movieId', 'title']] = movies[['movieId', 'title']]
    
    # Filtrar películas calificadas por el usuario
    movies_r = df_dum[df_dum['movieId'].isin(ratings_array)]
    
    # Eliminar columnas movieId y title
    movies_r = movies_r.drop(columns=['movieId', 'title'])
    movies_r["indice"] = 1  # Para usar group by y que quede en formato pandas tabla de centroide
    
    # Centroide o perfil del usuario
    centroide = movies_r.groupby("indice").mean()
    
    # Filtrar películas no leídas
    movies_ns = df_dum[~df_dum['movieId'].isin(ratings_array)]
    
    # Eliminar columnas movieId y title
    movies_ns = movies_ns.drop(columns=['movieId', 'title'])
    
    # Entrenar modelo 
    model = neighbors.NearestNeighbors(n_neighbors=10, metric='cosine')
    model.fit(movies_ns)
    dist, idlist = model.kneighbors(centroide)
    
    ids = idlist[0]  # Queda en un array anidado, para sacarlo
    recomend_m = movies.loc[ids][['title']]
    leidos = movies[movies['movieId'].isin(ratings_array)][['title', 'movieId']]
    
    return recomend_m

In [28]:
# Llamar a la función con el usuario seleccionado
recomendar(608)


,title
256,Austin Powers: The Spy Who Shagged Me (1999)
21,Broken Arrow (1996)
221,Back to the Future Part II (1989)
276,Being John Malkovich (1999)
309,Cast Away (2000)
279,Sleepy Hollow (1999)
49,Pulp Fiction (1994)
165,"Shining, The (1980)"
179,Star Trek II: The Wrath of Khan (1982)
305,Unbreakable (2000)


### Top 5 de las películas con el género más visto por el usuario

## 4 Sistema de recomendación filtro colaborativo

In [ ]:
# Obtener datos; se filtran los mayores a 0
rat = pd.read_sql('SELECT * FROM ratings_final WHERE rating > 0', conn)

# Definir escala
reader = Reader(rating_scale=(1, 5))  # Corregí el nombre de la variable 'Reader' por 'reader'

# Leer datos con surprise
data = Dataset.load_from_df(rat[['userId', 'movieId', 'rating']], reader)

# Modelos 
models = [KNNBasic(), KNNWithMeans(), KNNWithZScore(), KNNBaseline()] 
results = {}

# Probar modelos
for model in models:
    CV_scores = cross_validate(model, data, measures=["MAE", "RMSE"], cv=5, n_jobs=-1)
    
    result = pd.DataFrame.from_dict(CV_scores).mean(axis=0).\
             rename({'test_mae':'MAE', 'test_rmse': 'RMSE'})
    results[str(model).split("algorithms.")[1].split("object ")[0]] = result

performance_df = pd.DataFrame.from_dict(results).T
performance_df.sort_values(by='RMSE')

# Se seleccionó el KNNBasic porque tiene mejores métricas en "MAE Y RMSE"

# Definir la grilla de parámetros para la búsqueda de hiperparámetros
param_grid = {
    'sim_options': {
        'name': ['msd', 'cosine'],
        'min_support': [5],
        'user_based': [False, True]
    }
}

gridsearchKNNBasic = GridSearchCV(KNNBasic, param_grid, measures=['rmse'], cv=2, n_jobs=-1)

gridsearchKNNBasic.fit(data)

gridsearchKNNBasic.best_params["rmse"]
gridsearchKNNBasic.best_score["rmse"]
gs_model = gridsearchKNNBasic.best_estimator['rmse']

# Entrenar con todos los datos, y realizar predicciones con el modelo afinado
trainset = data.build_full_trainset()
model1 = gs_model.fit(trainset)
predset = trainset.build_anti_testset()
len(predset)
predictions = gs_model.test(predset)

# Crear base; películas no vistas por usuario + calificación predicha 
predictions_df = pd.DataFrame(predictions) 
predictions_df.shape
predictions_df.head()
predictions_df['r_ui'].unique()
predictions_df.sort_values(by='est', ascending=False)

# Función para recomendar 11 películas a un usuario específico
def recomendaciones(user_id, n_recomend=11):
    predictions_userID = predictions_df[predictions_df['uid'] == user_id].\
                    sort_values(by="est", ascending = False).head(n_recomend)

    recomendados = predictions_userID[['iid', 'est']]
    recomendados.to_sql('reco', conn, if_exists="replace")
    
    # Cambié 'movies_1' por 'movies_sel' según tus tablas
    mov = pd.read_sql('''SELECT a.*, b.title FROM reco a LEFT JOIN movies_sel b ON a.iid=b.movieId''', conn)
    return mov

# Obtener y mostrar las 11 películas recomendadas al usuario 50
peliculas = recomendaciones(user_id=50, n_recomend=11)
print(peliculas)
